In [1]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

In [20]:
business_df = pd.read_excel('../raw/business_data.xlsx')
variable_map = pd.read_excel('../data/map_viz_variable.xlsx')

In [3]:
business_df.head()

,m_start,m_end,m_today,m_username,m_deviceid,m_subscriberid,m_imei,m_phonenumber,_span_style_color_FF_finances_and_HR_span,i_econ_stop_business,...,__version__,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags,_index
0,2021-04-28T15:14:45.949+05:45,2021-04-28T15:33:20.591+05:45,2021-04-28,username not found,ee.humanitarianresponse.info:efUjl6IoNb0tnOVO,subscriberid not found,ee.humanitarianresponse.info:efUjl6IoNb0tnOVO,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,174444706,510712d3-b5f6-47e7-b3d2-9ae5f49e3b94,2021-04-28T09:47:31,NaN,[],submitted_via_web,NaN,NaN,36
1,2021-05-07T10:59:17.820+05:45,2021-05-07T11:21:20.855+05:45,2021-05-07,username not found,ee.humanitarianresponse.info:ivuI8g0MlCGor6Ft,subscriberid not found,ee.humanitarianresponse.info:ivuI8g0MlCGor6Ft,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,176510702,8ff407ec-6aab-46c8-bb5b-cc48219cc389,2021-05-07T05:36:20,NaN,[],submitted_via_web,NaN,NaN,109
2,2021-04-26T17:28:00.072+05:45,2021-04-26T17:41:14.457+05:45,2021-04-26,username not found,ee.humanitarianresponse.info:0cXJ5kEGM9avnhKH,subscriberid not found,ee.humanitarianresponse.info:0cXJ5kEGM9avnhKH,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,173876161,fd294dd2-9126-4feb-9b54-a0d41843f063,2021-04-26T11:57:05,NaN,[],submitted_via_web,NaN,NaN,8
3,2021-05-01T08:07:51.848+05:45,2021-05-01T08:45:08.379+05:45,2021-05-01,username not found,ee.humanitarianresponse.info:RyUETda2QuZoNbud,subscriberid not found,ee.humanitarianresponse.info:RyUETda2QuZoNbud,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,175156051,76b937cb-67fb-4892-ac32-5ec77e023929,2021-05-01T03:00:10,NaN,[],submitted_via_web,NaN,NaN,69
4,2021-05-16T19:46:28.056+05:45,2021-05-16T20:06:45.761+05:45,2021-05-16,username not found,ee.humanitarianresponse.info:CS5rQbmD3nN55q2y,subscriberid not found,ee.humanitarianresponse.info:CS5rQbmD3nN55q2y,phonenumber not found,NaN,2,...,vwGwixFVQLjhFdchggLZtd,178003908,d9cfe125-2298-41e8-bdd8-17e7262ebbdb,2021-05-16T14:21:47,NaN,[],submitted_via_web,NaN,NaN,110


In [4]:
business_map_df = business_df[['m_name_business', 'm_today', 'm_coodinates']]
business_map_df = business_map_df[business_map_df['m_coodinates']!=' ']

In [5]:
business_map_df['coordinates'] = business_map_df['m_coodinates'].apply(func= lambda x: (float(x.split(' ')[0]), float(x.split(' ')[1])))
business_map_df.drop('m_coodinates', axis=1, inplace=True)

In [6]:
business_map_df.head()

,m_name_business,m_today,coordinates
0,Shisha Lounge & Bar Pvt Ltd,2021-04-28,"(27.7153038584918, 85.3103238891712)"
1,Hotel Vinayak pvt ltd,2021-05-07,"(26.98806, 85.89996)"
2,chitwan forest resort,2021-04-26,"(27.58148, 84.49332)"
3,Suburb cafe,2021-05-01,"(27.684759, 85.312688)"
4,hostel nextdoor,2021-05-16,"(27.684944, 85.315922)"


In [26]:
selected_variables = variable_map[variable_map['selected']=='yes']
s_select_variables = selected_variables[selected_variables['input_type']=='single-select']['variable'].tolist()
multi_select_variables = selected_variables[selected_variables['input_type']=='multi-select']['variable'].tolist()

In [32]:
s_select_df = []
for i in s_select_variables:
    each_var_df = business_df[['m_name_business', 'm_today', 'm_coodinates']]
    each_var_df = each_var_df[each_var_df['m_coodinates']!=' ']
    each_var_df['coordinates'] = each_var_df['m_coodinates'].apply(func= lambda x: (float(x.split(' ')[0]), float(x.split(' ')[1])))
    each_var_df.drop('m_coodinates', axis=1, inplace=True)
    each_var_df['variable'] = i
    each_var_df['value'] = business_df[i]
    if len(s_select_df)==0:
        s_select_df = each_var_df
    else:
        s_select_df = pd.concat((s_select_df, each_var_df), axis=0)

In [34]:
s_select_df

,m_name_business,m_today,coordinates,variable,value
0,Shisha Lounge & Bar Pvt Ltd,2021-04-28,"(27.7153038584918, 85.3103238891712)",i_econ_stop_business,2.0
1,Hotel Vinayak pvt ltd,2021-05-07,"(26.98806, 85.89996)",i_econ_stop_business,2.0
2,chitwan forest resort,2021-04-26,"(27.58148, 84.49332)",i_econ_stop_business,2.0
3,Suburb cafe,2021-05-01,"(27.684759, 85.312688)",i_econ_stop_business,2.0
4,hostel nextdoor,2021-05-16,"(27.684944, 85.315922)",i_econ_stop_business,2.0
...,...,...,...,...,...
107,Thamel Seasons Hotel Pvt.ltd,2021-05-02,"(27.7179141297178, 85.3101209550944)",o_perm_stop_biz_future_retrn_trsm_biz,1.0
108,Hotel Business,2021-05-03,"(27.6632585317559, 85.4449016109137)",o_perm_stop_biz_future_retrn_trsm_biz,NaN
109,The freedom cafe restaurant & bar,2021-05-05,"(28.2209793, 83.9559105)",o_perm_stop_biz_future_retrn_trsm_biz,1.0
110,Tour and trekking and Expedition,2021-05-06,"(27.7153213473598, 85.3117415550944)",o_perm_stop_biz_future_retrn_trsm_biz,NaN


In [21]:
base_map = folium.Map(location=[27.7, 85.3240], zoom_start=13)
HeatMap(data=business_map_df['coordinates'], radius=9, max_zoom=13, ).add_to(base_map)
# base_map.save('kirana_heatmap.html')
base_map